First we need to check the version for the framework we built our model on,
For here it is scikit-learn, because we created,trained and predicted on an extra trees regressor and saved it as extra_trees_reg.bin

```pip freeze | grep scikit-learn```

In [1]:
import sklearn
sklearn.__version__

'1.3.2'

create a python environment using pipenv install ....then followed by necessary packages to install there

```pipenv install scikit-learn flask --python==version```

following installation start the shell with

```pipenv shell```

simillar to 
```conda activate env```



next for convenience shorten the prompt using PS1 = "> ", this changes the prompt to only "> " symbol with no text at all

Next we will creat predict.py to predict using input features but before that lets test it out

In [1]:
import pickle
import numpy as np

with open('web service/extra_trees_reg.bin', 'rb') as f_in:
    dv,etg= pickle.load(f_in)

In [2]:
etg

ExtraTreesRegressor(max_depth=20, n_estimators=300, n_jobs=4, verbose=True)

We can understand that it takes PUID and DOID as inputs and then transforms it into dicts using dict vectorizer, so we need to build out pipeline as 

prediction input ------> dictvectorizer -------> model -------------> predictions

In [3]:
def predict(features):
    X = dv.transform(features)
    preds = etg.predict(X)
    return preds

In [4]:
predict({"PULocationID": 75, "DOLocationID": 41,"PU_DO":'75_41', "trip_distance": 1.23})

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 300 out of 300 | elapsed:    0.1s finished


array([16.22276587])

Since the model works fine we will go on building the predict.py

Then we will create test.py


After running the model then we will further go about creating flask and stuff


//Next we will define the flask operation in predict.py


//Then we will run the flask app(predict.py)

//Then we will run the test.py

Now the flask is not recommended for production env, hence we will use gunicorn

syntax is  
```typescript
gunicorn --bind=0.0.0.0:9696 predict:app
```

## Docker


The next step is to build the docker image

// First we specify the items/commands in the Dockerfile
// Docker build will follow the instructions to create a docker image from the dockerfile

execute: 
```javascript
docker build -t name of the image:version_no .
```

never forget to specify the .

run using the following commmand

```javascript
docker run -it --rm -p 9696:9696 rid_duration_prediction_service:v1
```

--------------------------------------------------------------------------

# 4.3

Rebuild the model from scratch

In [1]:
import mlflow
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor

In [17]:

link = 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet'
def data_reader(link):
    df = pd.read_parquet(link)
    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds()/60)
    df['PU_DO'] = str(df['PULocationID']) + '_' + str(df['DOLocationID'])
    return df[["PULocationID","DOLocationID","duration","PU_DO"]]

In [18]:
df = data_reader(link)

In [20]:
df.head()

,PULocationID,DOLocationID,duration,PU_DO
0,166,143,11.016667,0 166\n1 24\n2 223\n3 ...
1,24,43,6.766667,0 166\n1 24\n2 223\n3 ...
2,223,179,6.333333,0 166\n1 24\n2 223\n3 ...
3,41,238,5.816667,0 166\n1 24\n2 223\n3 ...
4,41,74,5.966667,0 166\n1 24\n2 223\n3 ...
